In [70]:
# the cards that are highlighted have the following class (in hand)
# class = card zoomable highlight handy
# the cards in hand that are not highlighted are as follows:
# class = card zoomable handy
# cards on board that are not highlighted
# card zoomable mappy
# cards on board controlled by opponent
# card zoomable mappy opponent-controlled
# cards on board you control that are highlighted
# card zoomable highlight mappy
# choice of taking an action (e.g. attacking, discarding a card, summoning a card, etc.)
# action-choice
# this shows up when attacking an enemy, 
# class= "chooseable-space-overlay" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;"
# the background color when an empty tile is highlighted 
# class="space-placeholder" style="background-color: rgba(153, 204, 102, 0.8); cursor: pointer;
# the background color when a tile is not highlighted
# style="background-color: rgba(0, 0, 0, 0); cursor: default;
# the buttons on the left side, go to next phase, etc.
# button class="control-panel-content-actions-action"
# empty spaces 
# class="space-placeholder-container"
# cards that opponent controls when you can force them
# card zoomable highlight mappy opponent-controlled

In [71]:
# !pip install webdriver_manager

In [97]:
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
import re
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains
from ast import literal_eval
np.__version__

'1.19.5'

In [99]:
card_df = pd.read_csv('Summoner_Wars_Cards.csv', index_col=0)
card_df

,Card Name,Card Classes,Event Phase,Active Event,Abilities,Ability Text,Under,Cost,Strength,Life,Faction,Copies,Range
0,Tacullu,['Summoner'],NaN,False,['Mind Capture'],"['When this unit attacks an enemy unit, if the...",False,NaN,3.0,13.0,Breakers,1,Ranged
1,Mind Control,[],Summon Phase,False,NaN,NaN,False,0.0,NaN,NaN,Breakers,2,NaN
2,Deceiver,['Common'],NaN,False,"['Stupify', 'Engage']","['When an adjacent enemy attacks any card, if ...",False,1.0,1.0,4.0,Breakers,5,Ranged
3,Mind Witch,['Common'],NaN,False,['Mimic'],"['At the start of your Move Phase, this unit m...",False,1.0,3.0,2.0,Breakers,5,Ranged
4,Gulldune,['Champion'],NaN,False,['Telepathic Command'],"['After this unit attacks an enemy card, this ...",False,6.0,4.0,6.0,Breakers,1,Melee
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,Glory Caller,['Common'],NaN,False,['Call to Arms'],"['After summoning this unit, you may target a ...",False,1.0,1.0,4.0,Mountain Vargath,4,Melee
236,Slagborn Fighter,['Common'],NaN,False,['Forceful Attack'],"['After this unit attacks and damages a unit, ...",False,1.0,2.0,3.0,Mountain Vargath,4,Melee
237,Battle Leader,[],Move Phase,True,NaN,NaN,False,0.0,NaN,NaN,Mountain Vargath,2,NaN
238,Marshal Forces,[],Attack Phase,False,NaN,NaN,False,0.0,NaN,NaN,Mountain Vargath,2,NaN


In [93]:
card_df['Abilities'][0]

"['Mind Capture']"

In [100]:
# literal eval treats each element as a list rather than a string
ability_list = [j for i in card_df['Abilities'][card_df['Abilities'].notnull()] for j in literal_eval(i)]
ability_list

['Mind Capture',
 'Stupify',
 'Engage',
 'Mimic',
 'Telepathic Command',
 'Flight',
 'Levitate',
 'Greater Push',
 'Steadfast',
 'Far Shot',
 'Swift',
 'Push',
 'Sly',
 'Relentless',
 'Charge',
 'Blood Runes',
 'Imbued Strength',
 'Magic Junkie',
 'Relentless',
 'Feed the Eater',
 'Immobile',
 'Cling',
 'Climb',
 'Gang Up',
 'Raise the Dead',
 'Soulless',
 'Infect',
 'Soul Shift',
 'Life Drain',
 'Summoned By Fire',
 'Wrath',
 'Immolate',
 'Blood Fury',
 'Imbued Strength',
 'Cursed',
 'Structural Shift',
 'Frost Bolt',
 'Living Gate',
 'Mobile Structure',
 'Slow',
 'Momentum',
 'Ice Shards',
 'Greater Frost Bolt',
 'Cold Snap',
 'Trample',
 'Frost Axe',
 'Spirit Bond',
 'Imbued Strength',
 'Prepare',
 'Swift Shot',
 'Momentum',
 'Imbued Life',
 'Prepare',
 'Swift Shot',
 'Inspire',
 'Withdraw',
 'Imbued Strength',
 'Trample',
 'Imbued Speed',
 'Trample',
 'Charged',
 'Commune With Spirits',
 "The Citadel's Might",
 'Heal',
 'Engage',
 'Protect',
 'Divine Shield',
 'Radiant Shot',
 'Gui

In [101]:
unique_card_ability_list = []
for i in ability_list:
    if i not in unique_card_ability_list:
        unique_card_ability_list.append(i)
len(unique_card_ability_list)

211

In [102]:
battlefield_list = card_df['Card Name'][card_df['Life'].notnull()]
battlefield_list

0               Tacullu
2              Deceiver
3            Mind Witch
4              Gulldune
5               Gwalark
             ...       
232                Quen
233             Torodin
234                Varn
235        Glory Caller
236    Slagborn Fighter
Name: Card Name, Length: 164, dtype: object

In [103]:
active_event_list = card_df['Card Name'][card_df['Active Event'] == True]
active_event_list

10             Hypnotic Call
11        Perplexing Tempest
21                   Pile On
23               Unrelenting
25          Sacrificial Pyre
37                   Ice Ram
57       Chant of Entangling
59          Chant of Weaving
61             Holy Judgment
69          Guarding Spirits
71              Renewed Hope
73     Spirit of the Phoenix
81            Blinding Flare
83        Divine Retribution
85                 For Glory
93              Brutal Force
94                    Freeze
95               Primal Fury
97            Out of Shadows
107               Smoke Bomb
109           Gathering Song
118              Battle Song
119              Sky Assault
130              Stimshrooms
133                    Learn
141                  Insight
142          Psychic Assault
143                     Seek
155                Willpower
177           Glinting Speed
181               Coordinate
190         Seek the Horizon
191                  Slip By
201           Law of Liberty
202           

In [104]:
under_card_list = card_df['Card Name'][card_df['Under'] == True]
under_card_list

34     Hellforged Weapon
44             Ice Smith
105          Hand Cannon
106           Moto Boots
153       Hellforged Axe
154    Hellforged Chains
158           Wrench Rat
165       Mortar Upgrade
166     Thruster Upgrade
167       Turret Upgrade
Name: Card Name, dtype: object

In [105]:
full_card_list = card_df['Card Name']
full_card_list

0               Tacullu
1          Mind Control
2              Deceiver
3            Mind Witch
4              Gulldune
             ...       
235        Glory Caller
236    Slagborn Fighter
237       Battle Leader
238      Marshal Forces
239     Ramming Advance
Name: Card Name, Length: 240, dtype: object

In [106]:
# factions listed in release order
factions = ['Breakers', 'Cave Goblins', 'Fallen Kingdom', 'Polar Dwarves', 'Savanna Elves', 'Vanguards',
            'Phoenix Elves', 'Tundra Orcs', 'Cloaks', 'Skyspear Avians', 'Fungal Dwarves', 'Eternal Council',
            'Obsidian Dwarves', 'Sand Goblins', 'Shadow Elves', 'Wayfarers', 'High Elves', 'Swamp Orcs',
             'Crimson Order', 'Mountain Vargath']

In [107]:
unit_classes = ['Summoner', 'Structure', 'Common', 'Champion']

In [108]:
df = pd.read_excel('../sw_alt_password.xlsx')

In [109]:
password = df['password'][1]
email = df['Email'][1]

In [110]:



def start_game(singleplayer, faction, opposing_faction, email, password, driver):
    url = 'https://www.plaidhatgames.com/accounts/login/?next=/redirect-swo'
    driver.get(url)



    time.sleep(3)
    login = driver.find_element_by_id('id_login')
    login.click()
    login.send_keys(email)
    time.sleep(.5)
    pw = driver.find_element_by_id('id_password')
    pw.click()
    pw.send_keys(password)
    time.sleep(.05)
    button = driver.find_element_by_class_name('primaryAction')
    button.click()
    time.sleep(15)
    buttons = driver.find_elements_by_class_name('menu-button-text-container')
    if singleplayer==True:
        buttons[1].click()
        time.sleep(3)
        # change some settings
        settings = driver.find_element_by_class_name('settings-button-left')
        settings.click()
        time.sleep(1)
        # auto dismiss events (can cause issues otherwise)
        autodismiss_events = driver.find_element_by_id('autodismiss_events')
        autodismiss_events.click()
        time.sleep(.05)
        # turn on attack friendly
        attack_friendly = driver.find_element_by_id('attack_friendly')
        attack_friendly.click()
        time.sleep(.05)
        # turn off the sound
        sound = driver.find_elements_by_id('text-size-input')
        move = ActionChains(driver)
        move.click_and_hold(sound[0]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)
        move.click_and_hold(sound[1]).move_by_offset(-100, 0).release().perform()
        time.sleep(.05)

        back_button = driver.find_element_by_class_name('full-overlay-back-button-img')
        back_button.click()
        time.sleep(.5)
        buttons = driver.find_elements_by_class_name('menu-button-text-container')
        buttons[3].click()
        time.sleep(1)
        # select player faction 
        faction_use_button = driver.find_elements_by_xpath(f"//*[contains(text(), '{faction}')]/../../div[2]/div[1]")
        faction_use_button[0].click()
        time.sleep(.1)
        # select opposing faction
        faction_use_button = driver.find_elements_by_xpath(f"//*[contains(text(), '{opposing_faction}')]/../../div[2]/div[1]")
        faction_use_button[0].click()
    else:
        buttons[0].click()



    

In [111]:
# function that determines whether it's your turn
def is_turn(driver):
    time.sleep(.5)
    try:
        driver.find_element_by_class_name('control-panel-content-actions-action')
        return True
    except:
        return False

In [112]:
# driver = webdriver.Chrome(executable_path=r"C:\Users\steve\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
driver = webdriver.Chrome(executable_path=r"C:\\Users\\steve\\.wdm\\drivers\\chromedriver\\win64\\119.0.6045.105\\chromedriver.exe")
# use this when you need a new version of chromedriver
# driver = webdriver.Chrome(ChromeDriverManager().install())
start_game(singleplayer=True, faction='Tundra Orcs', opposing_faction='Polar Dwarves',
          email=email, password=password, driver=driver)
print(is_turn(driver))

False


In [ ]:
# driver.page_source

In [124]:
# returns outer html for each space on the map
def get_board_spaces(driver):
    all_spaces = []
    for col in range(6):
        single_col_spaces = []
        for row in range(8):
            space_html = driver.find_element_by_id(f'map-space-{col}-{row}').get_attribute('outerHTML')
            if 'space-placeholder' in space_html:
                friendly = 0
                occupied = False
                card_abilities = ['']
                card_classes = ['']
            else:
                occupied = True
                
                if 'opponent' in space_html:
                    friendly = -1
            
                else:
                    friendly = 1
                    
                card = driver.find_element_by_id(f'map-space-{col}-{row}')
                card.click()
                time.sleep(.05)
                               
    
                zoom_html = driver.find_element_by_class_name('zoom-card').get_attribute('outerHTML')
        
                
                
                
                if 'zoom-card-ability' in zoom_html:         
                    html_card_abilities = re.findall('<div class="zoom-card-ability-name">[^<]+</div>',zoom_html)
                    regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-ability-name">'), re.escape('</div>')))
                    card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
                    
                    html_card_classes = re.findall('<div class="zoom-card-extras-unit">[^<]+</div>',zoom_html)
                    regex_card_classes = re.compile('{}(.*){}'.format(re.escape('<div class="zoom-card-extras-unit">'), re.escape('</div>')))
                    card_classes = regex_card_classes.findall(html_card_classes[0])
#                     card_classes = card_classes[0]
                    
                    
                    if 'Structure' in zoom_html:
                        card_classes.append('Structure')
                    print(card_classes)
                    
                    
                else:
                    card_abilities = ['']
                    card_classes = ['Structure']
                    print(card_classes)

                elem = driver.find_element_by_id('zoom-card')
                ac = ActionChains(driver)
                ac.move_to_element(elem).move_by_offset(300, 400).click().perform()
                time.sleep(.05)
                               
            
            
            # card-name
            if 'card-name' in space_html:
                html_card_name = re.findall('alt=[^<]+><div class="card-name">',space_html)
                regex_card_name = re.compile('{}(.*){}'.format(re.escape('alt='), re.escape('><div class="card-name">')))
                card_name = regex_card_name.findall(html_card_name[0])
                card_name = card_name[0]
            else:
                card_name = ''
            
            # card-strength
            if 'card-strength' in space_html:
                html_card_strength = re.findall('<div class="card-strength">[^<]+</div>',space_html)
                regex_card_strength = re.compile('{}(.*){}'.format(re.escape('<div class="card-strength">'), re.escape('</div>')))
                card_strength = regex_card_strength.findall(html_card_strength[0])
                card_strength = card_strength[0]
            else:
                card_strength=0
            
            # card-life
            if 'card-life' in space_html:
                html_card_life = re.findall('<div class="card-life[^<]+</div>',space_html)
                regex_card_life = re.compile('{}(.*){}'.format(re.escape('>'), re.escape('</div>')))
                card_life = regex_card_life.findall(html_card_life[0])
                card_life = card_life[0]
            else:
                card_life=0
            # boost
            if 'boost' in space_html:
                html_card_boost = re.findall('<div class="boost">[^<]+</div>',space_html)
                regex_card_boost = re.compile('{}(.*){}'.format(re.escape('<div class="boost">'), re.escape('</div>')))
                boost = regex_card_boost.findall(html_card_boost[0])
                boost = boost[0]
            else:
                boost=0
            # card-abilities-ability
#             if 'card-abilities-ability' in space_html:
#                 html_card_abilities = re.findall('<div class="card-abilities-ability">[^<]+</div>',space_html)
#                 regex_card_abilities = re.compile('{}(.*){}'.format(re.escape('<div class="card-abilities-ability">'), re.escape('</div>')))
#                 card_abilities = [regex_card_abilities.findall(html_card_abilities[i])[0] for i in range(len(html_card_abilities))]
#             else:
#                 card_abilities = ['']
            # append to the column list
            single_col_spaces.append([occupied, friendly, card_name, card_strength, card_life, boost, card_abilities,
                                      card_classes, row, col])
        
        # append to columns to all_spaces so it is a 2 dimensional list (can maybe perform convolutions?)
        all_spaces.append(single_col_spaces)
            
    return all_spaces        
            

In [125]:
spaces = get_board_spaces(driver)
spaces

['Summoner']
['Common', 'Structure']
['Structure']
['Common', 'Structure']
['Common']
['Summoner']
['Common']
['Common']
['Structure']


[[[False, 0, '', 0, 0, 0, [''], [''], 0, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 0],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 0]],
 [[False, 0, '', 0, 0, 0, [''], [''], 0, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 1, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 2, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 3, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 4, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 5, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 6, 1],
  [False, 0, '', 0, 0, 0, [''], [''], 7, 1]],
 [[True,
   -1,
   '"Svara"',
   '3',
   '12',
   0,
   ['Structural Shift'],
   ['Summoner'],
   0,
   2],
  [True,
   -1,
   '"Ice Golem"',
   '2',
   '5',
   0,
   ['Living Gate', 'Mobile Structure', 'Slow'],
   ['Common', 'Structure'],
   1,
   2],
  [True, -1, '"gate"', 0,

In [86]:
def get_active_events(driver):
    try:
        active_html = driver.find_element_by_class_name('active-display-opponent').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        opponent_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        opponent_active_events = ['']
    try:
        active_html = driver.find_element_by_class_name('active-display-mine').get_attribute('outerHTML')
        html_active_events = re.findall('title=[^=]+',active_html)
        regex_active_events = re.compile('{}(.*){}'.format(re.escape('title="'), re.escape('" class')))
        friendly_active_events = [regex_active_events.findall(html_active_events[i])[0] for i in range(len(html_active_events))]
    except:
        friendly_active_events = ['']
    return opponent_active_events, friendly_active_events

In [88]:
opponent_active_events, friendly_active_events = get_active_events(driver)
opponent_active_events, friendly_active_events

([], ['Insight', 'Learn'])

In [122]:
# function to obtain amount of magic, cards in hand and deck
def get_resources(driver):
    user_frames = driver.find_elements_by_class_name('user-framed-stuff')
    enemy_frames = user_frames[0].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',enemy_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    enemy_magic = regex_magic.findall(html_magic[0])
    enemy_magic = enemy_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',enemy_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    enemy_hand = regex_hand.findall(html_hand[0])
    enemy_hand = enemy_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',enemy_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    enemy_draw = regex_draw.findall(html_draw[0])
    enemy_draw = enemy_draw[0]
    
    friendly_frames = user_frames[1].get_attribute('outerHTML')
    html_magic = re.findall('title="magic"><span>[^<]+</span>',friendly_frames)
    regex_magic = re.compile('{}(.*){}'.format(re.escape('title="magic"><span>'), re.escape('</span>')))
    friendly_magic = regex_magic.findall(html_magic[0])
    friendly_magic = friendly_magic[0]
    html_hand = re.findall('title="hand"><span>[^<]+</span>',friendly_frames)
    regex_hand = re.compile('{}(.*){}'.format(re.escape('title="hand"><span>'), re.escape('</span>')))
    friendly_hand = regex_hand.findall(html_hand[0])
    friendly_hand = friendly_hand[0]
    html_draw = re.findall('title="draw"><span>[^<]+</span>',friendly_frames)
    regex_draw = re.compile('{}(.*){}'.format(re.escape('title="draw"><span>'), re.escape('</span>')))
    friendly_draw = regex_draw.findall(html_draw[0])
    friendly_draw = friendly_draw[0]
    return [int(enemy_magic), int(enemy_hand), int(enemy_draw), int(friendly_magic), int(friendly_hand), int(friendly_draw)]


In [123]:
resources = get_resources(driver)
resources

[3, 5, 21, 3, 5, 25]

In [170]:
def get_hand(driver):  
    hand_buttons = driver.find_elements_by_class_name('framed-hand')
    # click on the second hand button (your own hand)
    hand_buttons[1].click()
    time.sleep(.05)
    hand_html = driver.find_element_by_class_name('summoner-wars-table-hand').get_attribute('outerHTML')
    html_card_names = re.findall('class="card-name">[^<]+</div>',hand_html)
    regex_card_names = re.compile('{}(.*){}'.format(re.escape('class="card-name">'), re.escape('</div>')))
    card_names = [regex_card_names.findall(html_card_names[i])[0] for i in range(len(html_card_names))]
    html_card_costs = re.findall('class="card-cost[^"]*"><span>[^<]+</span>',hand_html)
    regex_card_costs = re.compile('{}(.*){}'.format(re.escape('<span>'), re.escape('</span>')))
    card_costs = [int(regex_card_costs.findall(html_card_costs[i])[0]) for i in range(len(html_card_costs))]
    return card_names, card_costs

In [171]:
hand_card_names, hand_card_costs = get_hand(driver)
hand_card_names, hand_card_costs

(['Witch', 'Witch', 'gate', 'Splack', 'Vinemancer'], [1, 1, 0, 4, 1])

In [113]:
def get_discard(driver):
    discard_buttons = driver.find_elements_by_class_name('framed-discard')
    discard_buttons[1].click()
    time.sleep(.05)
    discard_html = driver.find_element_by_class_name('summoner-wars-table-discard').get_attribute('outerHTML')
    html_card_names = re.findall('class="card-name">[^<]+</div>',discard_html)
    regex_card_names = re.compile('{}(.*){}'.format(re.escape('class="card-name">'), re.escape('</div>')))
    card_names = [regex_card_names.findall(html_card_names[i])[0] for i in range(len(html_card_names))]
    return card_names

In [115]:
discard_card_names = get_discard(driver)
discard_card_names

['Shaman', 'Fighter', 'Fury']

In [116]:
def get_deck(driver):
    discard_buttons = driver.find_elements_by_class_name('framed-draw')
    discard_buttons[1].click()
    time.sleep(.05)
    deck_html = driver.find_element_by_class_name('summoner-wars-table-my-draw').get_attribute('outerHTML')
    html_card_names = re.findall('class="card-name">[^<]+</div>',deck_html)
    regex_card_names = re.compile('{}(.*){}'.format(re.escape('class="card-name">'), re.escape('</div>')))
    card_names = [regex_card_names.findall(html_card_names[i])[0] for i in range(len(html_card_names))]
    return card_names

In [118]:
deck_card_names = get_deck(driver)
deck_card_names

['Shonk',
 'Targan',
 'Shaman',
 'Shaman',
 'Shaman',
 'Smasher',
 'Smasher',
 'Smasher',
 'Charger',
 'Charger',
 'Charger',
 'Charger',
 'Fighter',
 'Fighter',
 'Fighter',
 'Fighter',
 'Brutal Force',
 'For Glory',
 'For Glory',
 'Freeze',
 'Freeze',
 'gate',
 'gate',
 'gate']

In [127]:
def get_history_log(driver):
    history_button = driver.find_elements_by_class_name('history-button-image')
    history_button[2].click()
    history_log_html = driver.find_element_by_class_name('full-log').get_attribute('outerHTML')
    html_history = re.findall('class="full-log-entry[^"]*"><span>[^<]+</span>',history_log_html)
    regex_history = re.compile('{}(.*){}'.format(re.escape('<span>'), re.escape('</span>')))
    history = [regex_history.findall(html_history[i])[0] for i in range(len(html_history))]
    return history

In [128]:
history = get_history_log(driver)
history

['Beginning of turn 1.',
 'Player 2 played the event Ice Ram.',
 'Player 2 summoned Ice Smith.',
 'Player 2 moved Ice Smith.',
 'Player 2 moved Ice Smith.',
 'Player 2 moved Svara.',
 'Player 2 moved Svara.',
 'Player 2 forced gate.',
 'Player 2 moved Frost Mage.',
 'Player 2 played the event Glacial Shift.',
 'Player 2 played the event Glacial Shift.',
 'Player 2 forced Ice Golem.',
 'Frost Mage received 1 damage.',
 'Player 2 forced Frost Mage.',
 'Player 2 forced gate.',
 'Frost Mage received 1 damage.',
 'Player 2 forced Frost Mage.',
 'Player 2 built a structure.',
 'Player 2 built a structure.',
 'Player 2 is attacking with Frost Mage, targeting Tundra Fighter.',
 'Player 2 rolls:  ',
 'Tundra Fighter received 1 damage.',
 'Player 2 is attacking with Ice Smith, targeting Frost Shaman.',
 'Player 2 rolls:  ',
 'Frost Shaman received 2 damage.',
 'Beginning of turn 2.',
 'Player 1 moved Frost Shaman.',
 'Player 1 moved Frost Shaman.',
 'Player 1 moved Grognack.',
 'Player 1 is atta

In [ ]:
def get_enemy_discards_remaining_cards(driver)

In [ ]:
# zoom-card-extras-event-phase
# added draw phase just to be safe and future proof in case they come out with a draw phase event 
event_phases = ['Summon Phase', 'Move Phase', 'Build Phase', 'Attack Phase', 'Magic Phase', 'Draw Phase']

In [ ]:
# for multiplayer where you don't get to choose opponent
def get_enemy_summoner(driver):

In [ ]:
# observation, reward, done, and info?

In [ ]:
# gets the current state of the board, cards in hand, discarded cards, active events, cards in deck, etc.
def get_board_state()

In [ ]:
# possible actions that can be taken at any given point
def get_available_actions()

In [ ]:
# choose an action, evaluating them?? (need to refresh on reinforcement learning here)
def evaluate_actions()